# Holt Winters Exponential Smoothing Model

In [1]:
# Imports
import sys
import os
from datetime import datetime
from datetime import date
import pandas as pd
import numpy as np
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
import matplotlib.pyplot as plt

# To import the main.py file
sys.path.append('../')
from python_files import main

# Getting all the data
confirmed_global, deaths_global, recovered_global, country_cases = main.collect_data()

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
pd.options.display.float_format = '{:.5f}'.format

In [4]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [25]:
def predict(country_name):
    # Getting the confirmed cases of that country
    cases = main.get_new_cases(country_name)
    
    # Removing the zero values 
    is_0 = cases['cases'] != 0
    cases = cases[is_0]
    
    # Making the training and test sets
    split_ratio = 0.80
    train_size = int(split_ratio * len(cases))
    train_df, test_df = cases.iloc[:train_size, :], cases.iloc[train_size:, :]
    train_df, test_df = pd.Series(train_df['cases'].values, train_df['date']), pd.Series(test_df['cases'].values, test_df['date'])
    cases = pd.Series(cases['cases'].values, cases['date'])
    
    # Model
    model = Holt(train_df, initialization_method = 'heuristic').fit(smoothing_level = 0.3, smoothing_trend = 2.6, optimized = True)
    forecast1 = model.forecast(len(test_df))
    
    print(mape(forecast1, test_df))

In [26]:
predict('India')

9.081015186205162


In [27]:
predict('Russia')

1.4730656405466918


In [29]:
def grid_search_holt(country_name, sl, st):
    # Getting the confirmed cases of that country
    cases = main.get_new_cases(country_name)
    
    # Removing the zero values 
    is_0 = cases['cases'] != 0
    cases = cases[is_0]
    
    # Making the training and test sets
    split_ratio = 0.80
    train_size = int(split_ratio * len(cases))
    train_df, test_df = cases.iloc[:train_size, :], cases.iloc[train_size:, :]
    train_df, test_df = pd.Series(train_df['cases'].values, train_df['date']), pd.Series(test_df['cases'].values, test_df['date'])
    cases = pd.Series(cases['cases'].values, cases['date'])
    
    # Model
    model = Holt(train_df, initialization_method = 'heuristic').fit(smoothing_level = sl, smoothing_trend = st, optimized = True)
    forecast1 = model.forecast(len(test_df))
    
    return (mape(forecast1, test_df))

In [30]:
%%time
# Custom grid search
sl = [i/10 for i in range(1, 11)]
st = [i/10 for i in range(1, 11)]
lowest = 100
sl_l = None
st_l = None

for sli in sl:
    for sti in st:
        ma = grid_search_holt('India', sli, sti)
        if ma <= lowest:
            lowest = ma
            sl_l = sli
            st_l = sti
print(lowest, sl_l, st_l)

6.4195715097104795 0.3 1.0
CPU times: user 3.13 s, sys: 3.99 ms, total: 3.13 s
Wall time: 3.14 s
